# Analysis on the auxiliary requirements for gasification and requirements for gas cleaning

In [1]:
# Fix sys path to root directory so locally defined packages can be found
import sys
sys.path.append('C:\\Users\\2270577A\\PycharmProjects\\PhD_LCA_TEA')  # fix path so I can import local functions etc.

In [2]:
import pickle

import pandas as pd
import numpy as np

from functions.general.utility import reject_outliers
from objects import triangular_dist_maker


### Reference 1:

In [3]:
# Synthetic natural gas production via allothermal biomass gasification
ref1 = "https://doi.org/10.1002/er.2933"
biomass_input = 500  # kW
aux_power = 10  # kW
aux_fraction = aux_power / biomass_input  # as fraction of biomass input. Note: Uses power not energy.
print("Auxiliary requirements as fraction of biomass input:", aux_fraction*100, "%")


Auxiliary requirements as fraction of biomass input: 2.0 %


### Reference 2:

In [4]:
# Exergy analysis of hydrogen production plants based on biomass gasification
ref2 = "https://doi.org/10.1016/j.ijhydene.2008.05.059"
# Battelle, Vaernamo, and FICFB systems used for reference

exergy_data = pd.DataFrame({"Battelle LTGC": {"Exergy fuel [kW]": 12226.71, "Exergy loss gas cleaning [kW]": 254.51},
                            "Battelle HTGC": {"Exergy fuel [kW]": 12226.71, "Exergy loss gas cleaning [kW]": 385.06},
                            "Vaernamo LTGC": {"Exergy fuel [kW]": 11762.67, "Exergy loss gas cleaning [kW]": 1546.21},
                            "Vaernamo HTGC": {"Exergy fuel [kW]": 11762.67, "Exergy loss gas cleaning [kW]": 618.76},
                            "FICFB LTGC": {"Exergy fuel [kW]": 11827.5, "Exergy loss gas cleaning [kW]": 343.52},
                            "FICFB HTGC": {"Exergy fuel [kW]": 11827.5, "Exergy loss gas cleaning [kW]": 172.74}
                            }).transpose()

exergy_data["Gas cleaning exergy fraction"] = exergy_data["Exergy loss gas cleaning [kW]"] / exergy_data["Exergy fuel [kW]"]

# Display data
exergy_data


,Exergy fuel [kW],Exergy loss gas cleaning [kW],Gas cleaning exergy fraction
Battelle LTGC,12226.71,254.51,0.020816
Battelle HTGC,12226.71,385.06,0.031493
Vaernamo LTGC,11762.67,1546.21,0.131451
Vaernamo HTGC,11762.67,618.76,0.052604
FICFB LTGC,11827.50,343.52,0.029044
FICFB HTGC,11827.50,172.74,0.014605


In [5]:
print("Auxiliary requirements as fraction of biomass input [%]:")
exergy_data["Gas cleaning exergy fraction"] * 100


Auxiliary requirements as fraction of biomass input [%]:


Battelle LTGC     2.081590
Battelle HTGC     3.149335
Vaernamo LTGC    13.145060
Vaernamo HTGC     5.260370
FICFB LTGC        2.904418
FICFB HTGC        1.460495
Name: Gas cleaning exergy fraction, dtype: float64

In [6]:
# Show mean requirements
print("Mean auxiliary requirements as fraction of biomass input before removing outliers:", np.mean(exergy_data["Gas cleaning exergy fraction"] * 100), "%")
print("Mean auxiliary requirements as fraction of biomass input after removing outliers:",np.mean(reject_outliers(exergy_data["Gas cleaning exergy fraction"] * 100, 2)), "%")


Mean auxiliary requirements as fraction of biomass input before removing outliers: 4.666877849076176 %
Mean auxiliary requirements as fraction of biomass input after removing outliers: 2.9712414683733748 %


### Summary
- In general auxiliary demands and demands for syngas cleaning are very low.
- Mostly in the range of 2-5% of energy contained in the biomass input.
- A single outlier is as high as 13%.
- It can be assumed that the required energy is in the form of electricity to operate pumps, compressors, etc.
- A triangular distributions defined by a lower, mode, and upper boundary of 1.5, 3, and 5% respectively may best represent the auxiliary requirements.

In [7]:
# Prepare output and store in pickle file
requirements_triangular = triangular_dist_maker(lower=0.015, mode=0.03, upper=0.05)

# Store results
with open(r"results\gasification_aux_demands_results", "wb") as f:
    pickle.dump(requirements_triangular, f)

# Also save to data folder
with open(r"C:\Users\2270577A\PycharmProjects\PhD_LCA_TEA\data\gasification_aux_demands_results", "wb") as f:
    pickle.dump(requirements_triangular, f)

In [8]:
# Also store data as toml file
import toml
from functions.general.utility import get_project_root

toml_output = {"electricity_dist_maker": {"lower": requirements_triangular.lower,
                                   "mode": requirements_triangular.mode,
                                   "upper": requirements_triangular.upper
                                   }
               }

project_root = get_project_root()
file_name = str(project_root) + r"\data\gasification_aux_electricity_demands_results.toml"
with open(file_name, "w") as toml_file:
    toml.dump(toml_output, toml_file)
